# vLLM 기본 튜토리얼

### 이 튜토리얼에서 배울 내용

- vLLM 설치 및 서버 시작
- OpenAI 클라이언트로 vLLM 사용하기
- 기본 텍스트 생성 및 채팅
- 스트리밍 응답 처리
- 배치 처리로 여러 요청 동시 실행
- 생성 파라미터 조정

## 1. 환경 설정

vLLM을 사용하기 위해서는 GPU가 있는 환경이 권장된다. CPU에서도 작동하지만 성능이 제한적이다.

### 설치

```bash
pip install vllm
```

### 서버 시작

터미널에서 다음 명령어로 vLLM 서버를 시작한다:
```bash
python -m vllm.entrypoints.openai.api_server --model openai/gpt-oss-20b --port 8001
```

In [9]:
# 필요한 라이브러리 설치
#%pip install openai vllm

In [1]:
from openai import OpenAI
import time
import sys
import os

## 2. OpenAI 클라이언트 설정

In [2]:
vllm_path = "llms"
sys.path.append(os.path.dirname(vllm_path))

from vllm import LLM, SamplingParams
import torch

use_mps = torch.backends.mps.is_available()
device_type = "mps" if use_mps else "cpu"

print(f"Using device: {device_type}")

/home/josh/anaconda3/envs/lecture/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 11-17 23:06:02 [__init__.py:216] Automatically detected platform cuda.
Using device: cpu


In [8]:
# vLLM 서버에 연결하는 OpenAI 클라이언트 생성
client = OpenAI(
    base_url="http://localhost:8001/v1",  # vLLM 서버 주소
    api_key="EMPTY"  # vLLM은 API 키가 필요없지만 클라이언트는 값을 요구한다
)

# 사용할 모델 이름
MODEL_NAME = "openai/gpt-oss-20b"

print("vLLM 클라이언트가 설정되었다")
print(f"서버 주소: base_url")
print(f"모델: {MODEL_NAME}")

vLLM 클라이언트가 설정되었다
서버 주소: base_url
모델: openai/gpt-oss-20b


## 3. 첫 번째 텍스트 생성

In [9]:
# 간단한 텍스트 생성
def generate_completion(prompt, max_tokens=100):
    """
    주어진 프롬프트로 텍스트를 생성한다
    
    Args:
        prompt: 입력 프롬프트
        max_tokens: 생성할 최대 토큰 수
    
    Returns:
        생성된 텍스트
    """
    response = client.completions.create(
        model=MODEL_NAME,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0.7
    )
    
    return response.choices[0].text

# 테스트
prompt = "Python 프로그래밍 언어는"
result = generate_completion(prompt)

print(f"프롬프트: {prompt}")
print(f"생성 결과: {result}")

프롬프트: Python 프로그래밍 언어는
생성 결과:  주로 과학, 공학, 데이터 과학 분야에서 사용됩니다. 이는 데이터 과학 분야에서 여러 분야의 문제를 해결할 수 있는 다양한 라이브러리와 도구들이 존재하기 때문입니다. 이를 통해 Python은 다양한 분야에서 활용됩니다.

Python은 다양한 분야에서 활용되는 이유는 주로 다음과 같은 요소들 때문입니다:

1. **다양한 라이브러리와 프레임워크**: Python은 과학, 공학,


## 4. 채팅 API 사용하기

In [10]:
def chat(messages, temperature=0.7, max_tokens=500):
    """
    메시지 리스트로 채팅을 진행한다
    
    Args:
        messages: 메시지 리스트
        temperature: 창의성 조절 파라미터
        max_tokens: 최대 생성 토큰 수
    
    Returns:
        어시스턴트의 응답
    """
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

# 단일 메시지 테스트
messages = [
    {"role": "user", "content": "머신러닝이란 무엇인가요?"}
]

response = chat(messages)
print(f"사용자: {messages[0]['content']}")
print(f"\n어시스턴트: {response}")

사용자: 머신러닝이란 무엇인가요?

어시스턴트: ### 머신러닝(Machine Learning)이란?

**머신러닝**은 컴퓨터가 데이터를 통해 스스로 패턴을 찾아내고, 그 패턴을 바탕으로 예측하거나 결정을 내리는 과학과 기술입니다.  
다시 말해, “사람이 직접 프로그래밍하지 않아도, 경험(데이터)을 통해 학습하고 개선되는 컴퓨터 프로그램”이라고 할 수 있습니다.

---

## 1. 머신러닝이 필요한 이유

| 문제 | 전통적 프로그래밍 방식 | 머신러닝 방식 |
|------|---------------------|--------------|
| **복잡한 패턴 인식** (음성, 이미지, 자연어 등) | 수작업으로 규칙을 정의 → 어려움 | 데이터를 통해 패턴을 스스로 학습 |
| **데이터 양이 많고 빠르게 변함** | 사람 수동으로 업데이트 → 비효율 | 모델이 자동으로 학습 주기를 조정 |
| **비선형 관계** (예: 주가, 의료 진단) | 선형/비선형 수식으로 어려움 | 비선형 모델(딥러닝 등) 활용 |

---

## 2. 머신러닝의 핵심 개념

| 개념 | 설명 |
|------|------|
| **데이터** | 모델이 학습하고 예측하는 원천. 입력(특징) + 라벨(정답) |
| **모델(알고리즘)** | 입력 → 출력을 연결하는 수학적 구조. 예: 선형 회귀, 결정 트리, 신경망 |
| **학습(Training)** | 데이터와 모델을 사용해 가중치(매개변수)를 최적화. 오차를 줄이려 함. |
| **평가(Evaluation)** | 학습되지 않은 테스트 데이터를 이용해 성


## 5. 시스템 메시지 활용

In [11]:
# 시스템 메시지를 포함한 채팅
messages = [
    {
        "role": "system",
        "content": "당신은 전문 프로그래밍 강사입니다. 초보자도 이해하기 쉽게 설명하고, 실용적인 예시를 제공합니다."
    },
    {
        "role": "user",
        "content": "for 루프에 대해 설명해주세요."
    }
]

response = chat(messages)
print("=== 시스템 메시지 활용 예제 ===")
print(f"시스템 역할: {messages[0]['content']}")
print(f"\n질문: {messages[1]['content']}")
print(f"\n답변: {response}")

=== 시스템 메시지 활용 예제 ===
시스템 역할: 당신은 전문 프로그래밍 강사입니다. 초보자도 이해하기 쉽게 설명하고, 실용적인 예시를 제공합니다.

질문: for 루프에 대해 설명해주세요.

답변: ## ‘for 루프’란?

프로그래밍에서 **루프**(반복문)는 “같은 일을 여러 번 반복하려면 어떻게 해야 할까?” 라는 질문에 답해 주는 구조입니다.  
‘for 루프’는 **“정해진 개수만큼 반복해라”** 라는 의미를 갖고 있어 가장 많이 쓰이는 반복문 중 하나입니다.  

> **핵심**  
> * **시작값** → 반복을 시작할 때의 값  
> * **종료조건** → 반복을 멈출 조건(보통 “이 값이 아니라면 계속”)  
> * **증감식** → 한 번 반복이 끝난 뒤 값이 어떻게 바뀔지 정의  

이 세 가지를 명확히 하면 “반복이 언제 종료되는지 정확히 알 수 있으므로 무한루프에 빠지지 않는다”는 장점이 있습니다.

---

## 1. Python 예시

Python에서는 `for` 루프가 가장 직관적으로 사용됩니다.  
가장 기본적인 형태는 **시퀀스(리스트, 문자열, 튜플 등)** 를 한 번씩 순회합니다.

```python
# 1번부터 5번까지 출력
for i in range(1, 6):   # range(시작, 끝+1)
    print(i)
```

`range(1, 6)` 은 1, 2, 3, 4, 5 를 순서대로 생성하는 이터레이터입니다.  
위 코드의 실행 결과는

```
1
2
3
4
5
```

### 리스트 순회하기

```python
fruits = ['사과', '바나나', '체리']
for fruit in fruits:
    print(fruit)
```

### 문자열 순회하기

```python
for ch in "hello


## 6. 대화 컨텍스트 유지하기

In [12]:
# 대화 히스토리 초기화
conversation = [
    {
        "role": "system",
        "content": "당신은 도움이 되는 AI 어시스턴트입니다."
    }
]

def add_message_and_respond(user_message):
    """
    사용자 메시지를 추가하고 응답을 받는다
    
    Args:
        user_message: 사용자 메시지
    
    Returns:
        어시스턴트 응답
    """
    # 사용자 메시지 추가
    conversation.append({
        "role": "user",
        "content": user_message
    })
    
    # 응답 생성
    assistant_response = chat(conversation)
    
    # 어시스턴트 응답 추가
    conversation.append({
        "role": "assistant",
        "content": assistant_response
    })
    
    return assistant_response

# 연속 대화 예제
print("=== 연속 대화 예제 ===")

# 첫 번째 대화
user_msg_1 = "저는 데이터 과학을 공부하고 있습니다."
response_1 = add_message_and_respond(user_msg_1)
print(f"\n사용자: {user_msg_1}")
print(f"어시스턴트: {response_1}")

# 두 번째 대화
user_msg_2 = "제가 무엇을 공부한다고 했죠?"
response_2 = add_message_and_respond(user_msg_2)
print(f"\n사용자: {user_msg_2}")
print(f"어시스턴트: {response_2}")

print(f"\n현재 대화 길이: {len(conversation)}개 메시지")

=== 연속 대화 예제 ===

사용자: 저는 데이터 과학을 공부하고 있습니다.
어시스턴트: 데이터 과학을 공부하고 계시다니 멋지네요!  
어떤 주제(예: 기계 학습, 통계, 시각화, 데이터 전처리 등)나 프로젝트에 대해 도움을 드릴까요? 혹은 지금 겪고 있는 어려움이나 궁금한 점이 있으면 말씀해 주세요. 😊

사용자: 제가 무엇을 공부한다고 했죠?
어시스턴트: 네, 앞서 말씀하신 내용은 “저는 데이터 과학을 공부하고 있습니다.”입니다.  
데이터 과학에 대해 더 깊이 알아보고 싶으신 특정 영역(예: 머신러닝, 통계, 시각화, 데이터 전처리 등)이 있으신가요? 필요하신 자료나 조언이 있으면 언제든 말씀해 주세요!

현재 대화 길이: 5개 메시지


## 7. 스트리밍 응답 받기

In [13]:
def chat_stream(messages, temperature=0.7, max_tokens=500):
    """
    스트리밍 방식으로 채팅 응답을 받는다
    
    Args:
        messages: 메시지 리스트
        temperature: 창의성 조절 파라미터
        max_tokens: 최대 생성 토큰 수
    
    Returns:
        완전한 응답 문자열
    """
    # 스트리밍 활성화
    stream = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        stream=True  # 스트리밍 활성화
    )
    
    # 전체 응답 수집
    full_response = ""
    
    # 스트림에서 청크를 받아 처리
    for chunk in stream:
        # 델타(증분) 내용 추출
        delta_content = chunk.choices[0].delta.content
        
        if delta_content:
            full_response += delta_content
            # 실시간으로 출력
            print(delta_content, end="", flush=True)
    
    print()  # 줄바꿈
    return full_response

# 스트리밍 테스트
print("=== 스트리밍 응답 예제 ===")
messages = [
    {"role": "user", "content": "딥러닝의 주요 개념 3가지를 설명해주세요."}
]

print("\n어시스턴트: ", end="")
response = chat_stream(messages)

=== 스트리밍 응답 예제 ===

**딥러닝의 핵심 개념 3가지**

| # | 개념 | 핵심 내용 | 예시 |
|---|------|-----------|------|
| 1 | **딥 뉴럴 네트워크(Deep Neural Network)** | 입력 → 여러 개의 은닉층(hidden layer) → 출력으로 구성된 인공신경망. 각 층은 **가중치**와 **편향**을 통해 입력을 변환하고, **활성화 함수(activation function)** 로 비선형성을 부여해 복잡한 함수 근사 가능. 층이 많을수록 “깊음(depth)”이 증가해 더 고차원적, 추상적인 특징을 학습. | 이미지 분류에서 ConvNet(합성곱 신경망) → Conv–ReLU–Pool–FC 등 10여 층으로 구성된 모델. |
| 2 | **역전파(Backpropagation) & 경사 하강법(Gradient Descent)** | 모델이 예측한 출력과 실제 정답(레이블)


## 8. 배치 처리로 여러 요청 동시 실행

In [26]:
import concurrent.futures

def process_single_request(prompt):
    """
    단일 요청을 처리한다
    
    Args:
        prompt: 입력 프롬프트
    
    Returns:
        프롬프트와 응답을 포함한 딕셔너리
    """
    messages = [{"role": "user", "content": prompt}]
    response = chat(messages, max_tokens=100)
    return {"prompt": prompt, "response": response}

# 여러 프롬프트 준비
prompts = [
    "Python의 장점은?",
    "JavaScript의 특징은?",
    "Java의 주요 용도는?",
    "C++이 사용되는 분야는?",
    "Go 언어의 특징은?"
]

print("=== 배치 처리 예제 ===")
print(f"\n{len(prompts)}개의 요청을 동시에 처리한다...\n")

# 시작 시간 기록
start_time = time.time()

# 병렬로 요청 처리
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # 모든 요청을 동시에 제출
    future_to_prompt = {executor.submit(process_single_request, prompt): prompt for prompt in prompts}
    
    # 완료된 순서대로 결과 수집
    results = []
    for future in concurrent.futures.as_completed(future_to_prompt):
        result = future.result()
        results.append(result)

# 종료 시간 기록
end_time = time.time()
elapsed_time = end_time - start_time


# 결과 출력
for i, result in enumerate(results, 1):
    print(f"[요청 {i}]")
    print(f"질문: {result['prompt']}")
    
    # 응답이 있는지 확인
    if result.get('error', False):
        print(f"답변: {result['response']}")
    else:
        # 응답 길이 확인
        response_text = result['response'] if result['response'] else "응답 없음"
        preview = response_text[:100] if len(response_text) > 100 else response_text
        print(f"답변: {preview}...")
    print()

print(f"총 처리 시간: {elapsed_time:.2f}초")
print(f"평균 처리 시간: {elapsed_time/len(prompts):.2f}초/요청")

=== 배치 처리 예제 ===

5개의 요청을 동시에 처리한다...

[요청 1]
질문: Java의 주요 용도는?
답변: 응답 없음...

[요청 2]
질문: JavaScript의 특징은?
답변: 응답 없음...

[요청 3]
질문: Python의 장점은?
답변: 응답 없음...

[요청 4]
질문: C++이 사용되는 분야는?
답변: 응답 없음...

[요청 5]
질문: Go 언어의 특징은?
답변: 응답 없음...

총 처리 시간: 0.66초
평균 처리 시간: 0.13초/요청


## 9. 생성 파라미터 조정

In [17]:
def chat_with_params(messages, temperature=0.7, top_p=0.9, 
                     max_tokens=200, presence_penalty=0.0):
    """
    상세한 파라미터로 채팅을 진행한다
    
    Args:
        messages: 메시지 리스트
        temperature: 랜덤성 조절 (0.0 ~ 2.0)
        top_p: 누적 확률 임계값 (0.0 ~ 1.0)
        max_tokens: 최대 생성 토큰 수
        presence_penalty: 주제 다양성 조절 (-2.0 ~ 2.0)
    
    Returns:
        어시스턴트 응답
    """
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        presence_penalty=presence_penalty
    )
    
    return response.choices[0].message.content

# 동일한 프롬프트로 다른 파라미터 테스트
test_messages = [
    {"role": "user", "content": "짧은 이야기를 만들어주세요."}
]

print("=== 파라미터 비교 예제 ===")

# 낮은 temperature (더 결정적)
print("\n[Temperature = 0.2 (결정적)]")
response_low = chat_with_params(test_messages, temperature=0.2)
print(response_low)

# 높은 temperature (더 창의적)
print("\n[Temperature = 1.5 (창의적)]")
response_high = chat_with_params(test_messages, temperature=1.5)
print(response_high)

=== 파라미터 비교 예제 ===

[Temperature = 0.2 (결정적)]
**제목: 마지막 잎사귀**

봄이 끝나갈 무렵, 작은 마을에 사는 소녀 민지는 매일 아침 창문을 열어 들이마시는 바람을 좋아했다. 어느 날, 그녀는 마당

[Temperature = 1.5 (창의적)]
모자 쓴 작은 트리끼리의 벗교부활전  

미지근한 아침 햇살 아래에서 상비량이 결짓하는 작은 사막이 낡


## 10. 토큰 사용량 확인

In [21]:
def chat_with_usage(messages):
    """
    토큰 사용량 정보를 포함하여 채팅한다
    
    Args:
        messages: 메시지 리스트
    
    Returns:
        응답 내용과 사용량 정보를 포함한 딕셔너리
    """
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        max_tokens=300
    )
    
    return {
        "content": response.choices[0].message.content,
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens
    }

# 토큰 사용량 확인
messages = [
    {
        "role": "system",
        "content": "당신은 간결하게 답변하는 AI입니다."
    },
    {
        "role": "user",
        "content": "클라우드 컴퓨팅이란 무엇인가요?"
    }
]

result = chat_with_usage(messages)

print("=== 토큰 사용량 분석 ===")
print(f"\n응답: {result['content']}")
print(f"\n토큰 사용량:")
print(f"  - 프롬프트 토큰: {result['prompt_tokens']}")
print(f"  - 생성 토큰: {result['completion_tokens']}")
print(f"  - 총 토큰: {result['total_tokens']}")

=== 토큰 사용량 분석 ===

응답: 클라우드 컴퓨팅은 인터넷을 통해 서버, 스토리지, 데이터베이스, 네트워킹 등 컴퓨터 자원을 “서비스” 형태로 제공하고, 사용자가 필요할 때만 사용하고 비용을 지불하는 모델입니다.

토큰 사용량:
  - 프롬프트 토큰: 97
  - 생성 토큰: 125
  - 총 토큰: 222


## 11. 실용적인 채팅 클래스

In [22]:
class VLLMChatbot:
    """
    vLLM을 사용하는 채팅봇 클래스다
    """
    
    def __init__(self, system_message=None, model=MODEL_NAME):
        """
        채팅봇을 초기화한다
        
        Args:
            system_message: 시스템 메시지 (선택사항)
            model: 사용할 모델 이름
        """
        self.model = model
        self.history = []
        self.total_tokens = 0
        
        # 시스템 메시지 설정
        if system_message:
            self.history.append({
                "role": "system",
                "content": system_message
            })
    
    def chat(self, user_message, temperature=0.7, max_tokens=500):
        """
        사용자 메시지를 받아 응답을 생성한다
        
        Args:
            user_message: 사용자 메시지
            temperature: 창의성 조절
            max_tokens: 최대 생성 토큰
        
        Returns:
            어시스턴트 응답
        """
        # 사용자 메시지 추가
        self.history.append({
            "role": "user",
            "content": user_message
        })
        
        # 응답 생성
        response = client.chat.completions.create(
            model=self.model,
            messages=self.history,
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        # 응답 추출
        assistant_message = response.choices[0].message.content
        
        # 어시스턴트 응답 추가
        self.history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        # 토큰 사용량 누적
        self.total_tokens += response.usage.total_tokens
        
        return assistant_message
    
    def get_history_length(self):
        """
        현재 대화 히스토리 길이를 반환한다
        """
        return len(self.history)
    
    def get_total_tokens(self):
        """
        누적 토큰 사용량을 반환한다
        """
        return self.total_tokens
    
    def clear_history(self):
        """
        대화 히스토리를 초기화한다
        """
        # 시스템 메시지만 유지
        system_messages = [msg for msg in self.history if msg["role"] == "system"]
        self.history = system_messages
        self.total_tokens = 0

# 채팅봇 사용 예제
print("=== vLLM 채팅봇 예제 ===")

# 채팅봇 생성
bot = VLLMChatbot(
    system_message="당신은 프로그래밍 전문가입니다. 명확하고 실용적인 조언을 제공합니다."
)

# 대화 1
response_1 = bot.chat("파이썬에서 리스트와 튜플의 차이는?")
print(f"\n사용자: 파이썬에서 리스트와 튜플의 차이는?")
print(f"봇: {response_1}")

# 대화 2
response_2 = bot.chat("어느 것을 언제 사용해야 하나요?")
print(f"\n사용자: 어느 것을 언제 사용해야 하나요?")
print(f"봇: {response_2}")

# 통계 출력
print(f"\n대화 통계:")
print(f"  - 총 메시지 수: {bot.get_history_length()}")
print(f"  - 누적 토큰 사용: {bot.get_total_tokens()}")

=== vLLM 채팅봇 예제 ===

사용자: 파이썬에서 리스트와 튜플의 차이는?
봇: ## 리스트(`list`) vs 튜플(`tuple`) – 핵심 차이점

| 구분 | 리스트 | 튜플 |
|------|--------|------|
| **문법** | `my_list = [1, 2, 3]` | `my_tuple = (1, 2, 3)`<br>단일 요소는 `a = (1,)`처럼 쉼표가 필요 |
| **가변성** | **가변(mutable)** – 생성 후에도 요소를 추가·삭제·대체 가능 | **불변(immutable)** – 한 번 만들면 내용이 바뀌지 않음 |
| **활용도** | 동적으로 크기가 바뀌는 컬렉션, 데이터를 추가/삭제할 때 | 고정된 데이터 집합, 함수에서 여러 값을 반환, dict key 등으로 활용 |
| **메모리/속도** | 약간 더 큰 메모리 사용, 요소 접근은 튜플보다 느림 | 메모리 사용이 적고, 인덱싱·반복이 빠름 |
| **메서드** | `append`, `extend`, `insert`, `remove`, `pop`, `sort`, `reverse` 등 | 메서드가 거의 없음 (인덱싱·슬라이싱만 가능) |
| **안전성** | 값이 바뀔

사용자: 어느 것을 언제 사용해야 하나요?
봇: None

대화 통계:
  - 총 메시지 수: 5
  - 누적 토큰 사용: 1523
